# FE009 - More OpenBabel Features
- Add more neighbors
- Add weight of neightbor atoms

In [1]:
import pandas as pd
import matplotlib.pylab as plt

In [2]:
import dask.dataframe as dd

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
%%time
angle_details = pd.read_csv('../data/openbabel/angle_details.csv')
atom_details = pd.read_csv('../data/openbabel/atom_details.csv')
bond_details = pd.read_csv('../data/openbabel/bond_details.csv')
closest = pd.read_csv('../data/openbabel/closest.csv')
distances = pd.read_csv('../data/openbabel/distances.csv')
molecule_details = pd.read_csv('../data/openbabel/molecule_details.csv')
torsion_details = pd.read_csv('../data/openbabel/torsion_details.csv')

CPU times: user 44.4 s, sys: 2.77 s, total: 47.1 s
Wall time: 47.2 s


In [5]:
tt = pd.concat([train, test], axis=0, sort=False)

In [6]:
tt.shape

(7163689, 6)

In [7]:
tt.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


# Atom Details

In [8]:
atom_details.head()

,molecule_name,atom_idx,atomic_mass,atomic_number,exact_mass,valence,spin_multiplicity
0,dsgdb9nsd_033805,0,14.0067,7,14.003074,1,0
1,dsgdb9nsd_033805,1,12.0107,6,12.000000,2,0
2,dsgdb9nsd_033805,2,12.0107,6,12.000000,4,0
3,dsgdb9nsd_033805,3,12.0107,6,12.000000,4,0
4,dsgdb9nsd_033805,4,12.0107,6,12.000000,4,0


In [9]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','atom_index_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'atom0_atomic_number',
                     'atomic_mass':'atom0_atomic_mass',
                     'valence':'atom0_valence',
                     'spin_multiplicity':'atom0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','atom_index_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'atom1_atomic_number',
                     'atomic_mass':'atom1_atomic_mass',
                     'valence':'atom1_valence',
                     'spin_multiplicity':'atom1_spin_multiplicity'})
tt.shape

CPU times: user 6.01 s, sys: 5.63 s, total: 11.6 s
Wall time: 11.7 s


# Angles

In [10]:
%%time
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','left_atom_idx','middle_atom_idx']).mean() \
              .reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'left_middle_average_angle'})

CPU times: user 9.14 s, sys: 6.16 s, total: 15.3 s
Wall time: 15.3 s


In [11]:
%%time
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','right_atom_idx','middle_atom_idx']).mean() \
              .reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','right_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'right_middle_average_angle'})

CPU times: user 8.84 s, sys: 6.33 s, total: 15.2 s
Wall time: 15.2 s


In [12]:
%%time
tt = pd.merge(tt,
         distances,
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','right_atom_idx'],
         how='left')\
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

CPU times: user 14.6 s, sys: 7.68 s, total: 22.3 s
Wall time: 22.3 s


# Closest Neighbors

In [13]:
# Fix bug with the indexes being off by one
if closest['closest'].min() == 1:
    closest['atom_index'] = closest['atom_index'] - 1
    for col in closest.columns:
        if 'close' in col:
            closest[col] = closest[col] - 1

In [14]:
%%time
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest',
                  '5th_closest', '6th_closest','7th_closest','8th_closest','9th_closest','10th_closest']],
        left_on=['molecule_name','atom_index_0'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_0',
                     '2nd_closest':'2nd_closest_to_0',
                     '3rd_closest':'3rd_closest_to_0',
                     '4th_closest':'4th_closest_to_0',
                     '5th_closest':'5th_closest_to_0',
                     '6th_closest':'6th_closest_to_0',
                     '7th_closest':'7th_closest_to_0',
                     '8th_closest':'8th_closest_to_0',
                     '9th_closest':'9th_closest_to_0',
                     '10th_closest':'10th_closest_to_0'})

CPU times: user 4.33 s, sys: 5.82 s, total: 10.1 s
Wall time: 10.2 s


In [15]:
%%time
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest',
                  '5th_closest', '6th_closest','7th_closest','8th_closest','9th_closest','10th_closest']],
        left_on=['molecule_name','atom_index_1'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_1',
                     '2nd_closest':'2nd_closest_to_1',
                     '3rd_closest':'3rd_closest_to_1',
                     '4th_closest':'4th_closest_to_1',
                     '5th_closest':'5th_closest_to_1',
                     '6th_closest':'6th_closest_to_1',
                     '7th_closest':'7th_closest_to_1',
                     '8th_closest':'8th_closest_to_1',
                     '9th_closest':'9th_closest_to_1',
                     '10th_closest':'10th_closest_to_1'})

CPU times: user 6 s, sys: 9.43 s, total: 15.4 s
Wall time: 15.5 s


In [16]:
tt.shape

(7163689, 40)

In [17]:
tt['is_closest_pair'] = tt['closest_to_0'] == tt['atom_index_1']
tt.shape

(7163689, 41)

# Distance to neighbors

## To atom0

In [18]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

CPU times: user 16.9 s, sys: 12.2 s, total: 29.1 s
Wall time: 29.2 s


In [19]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','2nd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_2nd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

CPU times: user 17.6 s, sys: 11.6 s, total: 29.1 s
Wall time: 29.2 s


In [20]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','3rd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_3rd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','4th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_4th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','5th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_5th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','6th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_6th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','7th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_7th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

CPU times: user 1min 56s, sys: 1min 12s, total: 3min 8s
Wall time: 3min 9s


In [21]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','8th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_8th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','9th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_9th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','10th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_10th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

CPU times: user 1min 27s, sys: 58.8 s, total: 2min 26s
Wall time: 2min 32s


## To atom1

In [23]:
tt.shape

(7163689, 61)

In [24]:
tt.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom0_atomic_mass,atom0_atomic_number,exact_mass_x,atom0_valence,atom0_spin_multiplicity,atom1_atomic_mass,atom1_atomic_number,exact_mass_y,atom1_valence,atom1_spin_multiplicity,left_middle_average_angle,right_middle_average_angle,distance,is_bond,closest_to_0,2nd_closest_to_0,3rd_closest_to_0,4th_closest_to_0,5th_closest_to_0,6th_closest_to_0,7th_closest_to_0,8th_closest_to_0,9th_closest_to_0,10th_closest_to_0,closest_to_1,2nd_closest_to_1,3rd_closest_to_1,4th_closest_to_1,5th_closest_to_1,6th_closest_to_1,7th_closest_to_1,8th_closest_to_1,9th_closest_to_1,10th_closest_to_1,is_closest_pair,distance_closest_to_0,is_bond_closest_to_0,distance_2nd_closest_to_0,is_bond_2nd_closest_to_0,distance_3rd_closest_to_0,is_bond_3rd_closest_to_0,distance_4th_closest_to_0,is_bond_4th_closest_to_0,distance_5th_closest_to_0,is_bond_5th_closest_to_0,distance_6th_closest_to_0,is_bond_6th_closest_to_0,distance_7th_closest_to_0,is_bond_7th_closest_to_0,distance_8th_closest_to_0,is_bond_8th_closest_to_0,distance_9th_closest_to_0,is_bond_9th_closest_to_0,distance_10th_closest_to_0,is_bond_10th_closest_to_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1.00794,1,1.007825,1,0,12.01070,6,12.000000,4,0,NaN,NaN,1.091953,True,0,2,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3,4,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,True,1.091953,True,1.78312,False,1.783147,False,1.783157,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,1.00794,1,1.007825,1,0,1.00794,1,1.007825,1,0,NaN,NaN,1.783120,False,0,2,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,False,1.091953,True,1.78312,False,1.783147,False,1.783157,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,1.00794,1,1.007825,1,0,1.00794,1,1.007825,1,0,NaN,NaN,1.783147,False,0,2,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,False,1.091953,True,1.78312,False,1.783147,False,1.783157,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,1.00794,1,1.007825,1,0,1.00794,1,1.007825,1,0,NaN,NaN,1.783157,False,0,2,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,3,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,False,1.091953,True,1.78312,False,1.783147,False,1.783157,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.00794,1,1.007825,1,0,12.01070,6,12.000000,4,0,NaN,NaN,1.091952,True,0,1,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3,4,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,True,1.091952,True,1.78312,False,1.783148,False,1.783158,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', 'closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '2nd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_2nd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '3rd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_3rd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '4th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_4th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)

CPU times: user 2min 3s, sys: 1min 20s, total: 3min 23s
Wall time: 3min 24s


In [26]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '5th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_5th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '6th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_6th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '7th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_7th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)

CPU times: user 2min 3s, sys: 1min 13s, total: 3min 16s
Wall time: 3min 16s


In [27]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '8th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_8th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '9th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_9th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '10th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_10th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt.shape

CPU times: user 2min 32s, sys: 1min 29s, total: 4min 2s
Wall time: 4min 2s


# Atom Details of Neighbors

In [28]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'closest_to_0_atomic_number',
                     'atomic_mass':'closest_to_0_atomic_mass',
                     'valence':'closest_to_0_valence',
                     'spin_multiplicity':'closest_to_0_spin_multiplicity'})
tt.shape

CPU times: user 46.4 s, sys: 32.6 s, total: 1min 18s
Wall time: 1min 19s


In [29]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','2nd_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'2nd_closest_to_0_atomic_number',
                     'atomic_mass':'2nd_closest_to_0_atomic_mass',
                     'valence':'2nd_closest_to_0_valence',
                     'spin_multiplicity':'2nd_closest_to_0_spin_multiplicity'})
tt.shape

CPU times: user 48 s, sys: 35.7 s, total: 1min 23s
Wall time: 1min 23s


In [30]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','3rd_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'3rd_closest_to_0_atomic_number',
                     'atomic_mass':'3rd_closest_to_0_atomic_mass',
                     'valence':'3rd_closest_to_0_valence',
                     'spin_multiplicity':'3rd_closest_to_0_spin_multiplicity'})
tt.shape

CPU times: user 51.4 s, sys: 39 s, total: 1min 30s
Wall time: 1min 30s


In [31]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','4th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'4th_closest_to_0_atomic_number',
                     'atomic_mass':'4th_closest_to_0_atomic_mass',
                     'valence':'4th_closest_to_0_valence',
                     'spin_multiplicity':'4th_closest_to_0_spin_multiplicity'})
tt.shape

CPU times: user 54.2 s, sys: 45.3 s, total: 1min 39s
Wall time: 1min 39s


In [32]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','5th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'5th_closest_to_0_atomic_number',
                     'atomic_mass':'5th_closest_to_0_atomic_mass',
                     'valence':'5th_closest_to_0_valence',
                     'spin_multiplicity':'5th_closest_to_0_spin_multiplicity'})

CPU times: user 56.4 s, sys: 47.2 s, total: 1min 43s
Wall time: 1min 43s


In [33]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','6th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'6th_closest_to_0_atomic_number',
                     'atomic_mass':'6th_closest_to_0_atomic_mass',
                     'valence':'6th_closest_to_0_valence',
                     'spin_multiplicity':'6th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','7th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'7th_closest_to_0_atomic_number',
                     'atomic_mass':'7th_closest_to_0_atomic_mass',
                     'valence':'7th_closest_to_0_valence',
                     'spin_multiplicity':'7th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','8th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'8th_closest_to_0_atomic_number',
                     'atomic_mass':'8th_closest_to_0_atomic_mass',
                     'valence':'8th_closest_to_0_valence',
                     'spin_multiplicity':'8th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','9th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'9th_closest_to_0_atomic_number',
                     'atomic_mass':'9th_closest_to_0_atomic_mass',
                     'valence':'9th_closest_to_0_valence',
                     'spin_multiplicity':'9th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','10th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'10th_closest_to_0_atomic_number',
                     'atomic_mass':'10th_closest_to_0_atomic_mass',
                     'valence':'10th_closest_to_0_valence',
                     'spin_multiplicity':'10th_closest_to_0_spin_multiplicity'})

MemoryError: 

In [34]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'closest_to_1_atomic_number',
                     'atomic_mass':'closest_to_1_atomic_mass',
                     'valence':'closest_to_1_valence',
                     'spin_multiplicity':'closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','2nd_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'2nd_closest_to_1_atomic_number',
                     'atomic_mass':'2nd_closest_to_1_atomic_mass',
                     'valence':'2nd_closest_to_1_valence',
                     'spin_multiplicity':'2nd_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','3rd_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'3rd_closest_to_1_atomic_number',
                     'atomic_mass':'3rd_closest_to_1_atomic_mass',
                     'valence':'3rd_closest_to_1_valence',
                     'spin_multiplicity':'3rd_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','4th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'4th_closest_to_1_atomic_number',
                     'atomic_mass':'4th_closest_to_1_atomic_mass',
                     'valence':'4th_closest_to_1_valence',
                     'spin_multiplicity':'4th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','5th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'5th_closest_to_1_atomic_number',
                     'atomic_mass':'5th_closest_to_1_atomic_mass',
                     'valence':'5th_closest_to_1_valence',
                     'spin_multiplicity':'5th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','6th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'6th_closest_to_1_atomic_number',
                     'atomic_mass':'6th_closest_to_1_atomic_mass',
                     'valence':'6th_closest_to_1_valence',
                     'spin_multiplicity':'6th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','7th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'7th_closest_to_1_atomic_number',
                     'atomic_mass':'7th_closest_to_1_atomic_mass',
                     'valence':'7th_closest_to_1_valence',
                     'spin_multiplicity':'7th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','8th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'8th_closest_to_1_atomic_number',
                     'atomic_mass':'8th_closest_to_1_atomic_mass',
                     'valence':'8th_closest_to_1_valence',
                     'spin_multiplicity':'8th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','9th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'9th_closest_to_1_atomic_number',
                     'atomic_mass':'9th_closest_to_1_atomic_mass',
                     'valence':'9th_closest_to_1_valence',
                     'spin_multiplicity':'9th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','10th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'10th_closest_to_1_atomic_number',
                     'atomic_mass':'10th_closest_to_1_atomic_mass',
                     'valence':'10th_closest_to_1_valence',
                     'spin_multiplicity':'10th_closest_to_1_spin_multiplicity'})

MemoryError: 

## Torsion Details

In [ ]:
%%time
tt = pd.merge(tt,
        torsion_details.groupby(['molecule_name','2left_atom_idx','left_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
        left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','left_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','left_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftleft_mean',
                     'min': 'tor_ang_2leftleft_min',
                    'max': 'tor_ang_2leftleft_max',
                    'count': 'tor_ang_2leftleft_count'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         torsion_details.groupby(['molecule_name','2left_atom_idx','right_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','right_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftright_mean',
                     'min': 'tor_ang_2leftright_min',
                     'max': 'tor_ang_2leftright_max',
                     'count': 'tor_ang_2leftright_count'})
tt.shape

In [ ]:
molecule_details.head()

In [ ]:
%%time
tt = pd.merge(tt,
        molecule_details.drop('num_residues', axis=1),
         left_on=['molecule_name'],
         right_on=['molecule_name'],
         how='left')

In [ ]:
test.head()

In [ ]:
test_FE009 = tt.sort_values('id').loc[tt['scalar_coupling_constant'].isnull()].reset_index(drop=True)
train_FE009 = tt.sort_values('id').loc[~tt['scalar_coupling_constant'].isnull()].reset_index(drop=True)

In [ ]:
test_FE009.shape

In [ ]:
test.shape

In [ ]:
train_FE009.shape

In [ ]:
train.shape

In [ ]:
bool_cols = [col for col in train_FE009.columns if 'is_bond_' in col]
test_FE009[bool_cols] = test_FE009[bool_cols].fillna(False)
train_FE009[bool_cols] = train_FE009[bool_cols].fillna(False)

In [ ]:
test_FE009.to_parquet('../data/FE009_test_pandas.parquet')

In [ ]:
train_FE009.to_parquet('../data/FE009_train_pandas.parquet')

In [ ]:
[col for col in test_FE009.columns]